In [ ]:
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np
from physnetjax.model import EF
from physnetjax.loss import dipole_calc
from  physnetjax.training import train_model # from model import dipole_calc
from physnetjax.data import prepare_batches, prepare_datasets
import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
import jax
import jax.numpy as jnp
import flax.linen as nn
import networkx as nx
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np
from physnetjax.model import EF
from physnetjax.loss import dipole_calc
from  physnetjax.training import train_model # from model import dipole_calc
from physnetjax.data import prepare_batches, prepare_datasets
import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
# from physnetjax.analysis import *
import polars as pl
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorflow.python.summary.summary_iterator import summary_iterator

In [2]:
from openqdc.datasets import Spice
from physnetjax.savepad import * 

NATOMS = 96

ds = Spice(
    energy_unit="ev",
    distance_unit="ang",
    # array_format = "jax"
)
ds.read_preprocess()
datadicts = [dict(ds[_]) for _ in ds.subsample(5000)]
output1 = process_in_memory(datadicts, max_atoms=NATOMS)
datadicts = [dict(ds[_]) for _ in ds.subsample(500)]
output2 = process_in_memory(datadicts, max_atoms=NATOMS)

2024-12-09 07:29:45.979 | INFO     | openqdc.datasets.base:read_preprocess:435 - Reading preprocessed data.
2024-12-09 07:29:45.980 | INFO     | openqdc.datasets.base:read_preprocess:436 - Dataset spice with the following units:
                     Energy: hartree,
                     Distance: bohr,
                     Forces: hartree/bohr
2024-12-09 07:29:47.351 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded atomic_inputs with shape (33175288, 5), dtype float32
2024-12-09 07:29:47.351 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded position_idx_range with shape (1110165, 2), dtype int32
2024-12-09 07:29:47.352 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded energies with shape (1110165, 1), dtype float64
2024-12-09 07:29:47.352 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded forces with shape (33175288, 3, 1), dtype float32
2024-12-09 07:29:47.353 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded n_atom

In [3]:
type(datadicts)

list

In [4]:
data_key, train_key = jax.random.split(
    jax.random.PRNGKey(42), 2)

2024-12-09 07:29:52.543238: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
from physnetjax.model import EF
from physnetjax.training import train_model 


In [ ]:
DEFAULT_DATA_KEYS = ["Z", "R", "D", "E", "F", "N"]

model = EF(natoms=NATOMS, charges=False, zbl=False)

params = train_model(
    train_key,
    model,
    output1,
    output2,
    num_epochs=int(1e6),
    learning_rate=0.001,
    energy_weight=NATOMS,
    #charges_weight=1,
    #forces_weight=100,
    schedule_fn="constant",
    optimizer="amsgrad",
    batch_size=10,
    num_atoms=NATOMS,
    data_keys=DEFAULT_DATA_KEYS,
   # restart="/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-ecdfaea0-2902-403e-8f44-27aba18a1c44/",
    print_freq=1,
    objective="valid_loss",
    best=1e6,
    )

Training Routine
Optimizer:
  init: <function chain.<locals>.init_fn at 0x14e284fe7560> init_fn
  update: <function chain.<locals>.update_fn at 0x14e284fe72e0> update_fn
Transform:
  init: <function reduce_on_plateau.<locals>.init_fn at 0x14e284fe7740> init_fn
  update: <function reduce_on_plateau.<locals>.update_fn at 0x14e284fe7ba0> update_fn
Schedule_fn: <function constant_schedule.<locals>.<lambda> at 0x14e1aaf9cc20> <lambda>


                                        PhysNetJax Training Initialization                                         
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Learning Rate ┃ Energy Weight ┃ Forces Weight    ┃ Dipole Weight    ┃ Charges Weight   ┃ Batch Size ┃ Num Atoms ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0.001         │ 96            │ 52.917721056384… │ 27.211386024367… │ 14.399645351950… │ 10         │ 96        │
└───────────────┴───────────────┴──────────────────┴──────────────────┴──────────────────┴────────────┴───────────┘

                                        PhysNetJax Training Initialization                                         
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃         ┃         ┃         ┃      ┃         ┃         ┃         ┃        ┃ Train   ┃ Valid  ┃         ┃        ┃
┃         ┃         ┃ Print   ┃      ┃         ┃         ┃ Data    ┃ Ckpt   ┃ Data    ┃ Data   ┃         ┃        ┃
┃ Restart ┃ Conver… ┃ Freq    ┃ Name ┃ Best    ┃ Object… ┃ Keys    ┃ Dir    ┃ Keys    ┃ Keys   ┃ Object… ┃ Saving ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ False   │ {'ener… │ 1       │ test │ 100000… │ valid_… │ ['Z',   │ /pche… │ dict_k… │ dict_… │ valid_… │ Saving │
│         │ 23.060… │         │      │         │         │ 'R',    │        │ 'N',    │ 'N',   │         │ a      │
│         │ 'force… │         │      │         │         │ 'D',    │        │ 'R',    │ 'R',   │         │ resta… │
│         │ 23.060… │         │      │         │         │ 'E',    │        │ 'F',    │ 'F',   │         │ file   │
│         │         │         │      │         │         │ 'F',    │        │ 'E'])   │ 'E'])  │         │ each   │
│         │         │         │      │         │         │ 'N']    │        │         │        │         │ time   │
│         │         │         │      │         │         │         │        │         │        │         │ the    │
│         │         │         │      │         │         │         │        │         │        │         │ valid… │
│         │         │         │      │         │         │         │        │         │        │         │ impro… │
└─────────┴─────────┴─────────┴──────┴─────────┴─────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┘

model EF(
    # attributes
    features = 32
    max_degree = 3
    num_iterations = 2
    num_basis_functions = 16
    cutoff = 6.0
    max_atomic_number = 118
    charges = False
    natoms = 96
    total_charge = 0
    n_res = 3
    zbl = False
    debug = False
)
Traced<ShapedArray(int32[960], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-c7b5eba7-c942-423c-8e06-bd1312ad087b/epoch-3
best!
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-c7b5eba7-c942-423c-8e06-bd1312ad087b/epoch-4
best!
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-c7b5eba7-c942-423c-8e06-bd1312ad087b/epoch-5
best!
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-c7b5eba7-c942-423c-8e06-bd1312ad087b/epoch-6
best!
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-c7b5eba7-c942-423c-8e06-bd1312ad087b/epoch-7
best!


In [ ]:
files = ["/pchem-data/meuwly/boittier/home/cf3criegee_27887.npz"]
train_data, valid_data = prepare_datasets(data_key, 25877, 2000,
                                          files, 
                                          clip_esp=False, natoms=8, clean=False)

In [383]:
train_data["R"].shape, output["R"].shape

((25877, 8, 3), (10, 100, 3))

In [384]:
train_data["Z"].shape, output["Z"].shape

((25877, 8), (10, 100))

In [413]:
train_data["N"].shape, output["N"].shape

((25877, 1), (10, 1))

In [390]:
train_data.keys()

dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])

In [412]:
DEFAULT_DATA_KEYS = ["Z", "R", "D", "E", "F", "N"]

params = train_model(
    train_key,
    model,
    train_data,
    train_data,
    num_epochs=int(1e6),
    learning_rate=0.001,
    energy_weight=NATOMS,
    #charges_weight=1,
    #forces_weight=100,
    schedule_fn="constant",
    optimizer="amsgrad",
    batch_size=1,
    num_atoms=8,
    data_keys=DEFAULT_DATA_KEYS,
#    restart=restart,
    print_freq=1,
    objective="valid_loss",
    best=1e6,
    )

Training Routine
Optimizer:
  init: <function chain.<locals>.init_fn at 0x14c1b18432e0> init_fn
  update: <function chain.<locals>.update_fn at 0x14c1b18431a0> update_fn
Transform:
  init: <function reduce_on_plateau.<locals>.init_fn at 0x14c1b1843240> init_fn
  update: <function reduce_on_plateau.<locals>.update_fn at 0x14c1b1843880> update_fn
Schedule_fn: <function constant_schedule.<locals>.<lambda> at 0x14c1b1841300> <lambda>
learning_rate: 0.001
energy_weight: 200
forces_weight: 52.91772105638412
dipole_weight: 27.211386024367243
charges_weight: 14.399645351950548
batch_size: 1
num_atoms: 8
restart: False
conversion: {'energy': 23.060548012069496, 'forces': 23.060548012069496}
print_freq: 1
name: test
best: 1000000.0
objective: valid_loss
data_keys: ['Z', 'R', 'D', 'E', 'F', 'N']
ckpt_dir: /pchem-data/meuwly/boittier/home/pycharmm_test/ckpts
Train data keys: dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])
Valid data keys: dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])
Objective: va

TypeError: 'NoneType' object is not subscriptable